In [1]:
import os
import uuid
import numpy as np
import pandas as pd
import itertools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from isadream import demos, demo_path, io
from isadream.models import utils

In [4]:
x_groups=(('Total Aluminate Concentration', ('Molar',), ("Al",)),
          ('Counter Ion Concentration', ('Molar',),
           ("Na+", "Li+", "Cs+", "K+")),
          ('Counter Ion', ('Species',), ("Na+", "Li+", "Cs+", "K+",)),
          ('Base Concentration', ('Molar',), ("OH-",)))

y_groups=(('27 Al ppm', ('ppm',), ("Al",)),)

test_groups = x_groups + y_groups

In [9]:
demo_json = io.read_idream_json(os.path.join(demo_path, demos["SIPOS_NMR_V2"]))
node = io.parse_node_json(demo_json)

In [41]:
data, metadata = io.collate_node(node, test_groups)
# display(data)
# type(data)
# display(data[0])
# data[("Counter Ion Concentration")]
# data.iloc[[0, 1], :]
data
# data.swaplevel(0,1, axis=1).xs("data", axis=1)
# data.swaplevel(0,1, axis=1).xs("metadata", axis=1).iloc[[0, 1], :]

Total Aluminate Concentration  \
                            data   
0                          0.005   
1                          0.005   
2                          0.005   
3                          0.005   
4                          0.005   
5                          0.005   
6                          0.005   
7                          0.005   
8                          0.005   
9                          0.005   
10                         0.005   
11                         0.005   
12                         0.005   
13                         0.005   
14                         0.005   
15                         0.005   
16                         0.005   
17                         0.005   
18                         0.005   
19                         0.005   
20                         0.005   
21                         0.005   
22                         0.050   
23                         0.050   
24                         0.050   
25                         0.050   
26                         0.050   
27                         0.050   
28                         0.050   
29                         0.050   
..                           ...   
61                         0.800   
62                         0.800   
63                         0.800   
64                         1.200   
65                         1.200   
66                         1.200   
67                         1.200   
68                         1.200   
69                         1.200   
70                         1.200   
71                         1.200   
72                         1.200   
73                         1.200   
74                         1.200   
75                         2.000   
76                         2.000   
77                         2.000   
78                         2.000   
79                         2.000   
80                         2.000   
81                         2.000   
82                         2.000   
83                         2.000   
84                         3.000   
85                         3.000   
86                         3.000   
87                         3.000   
88                         3.000   
89                         3.000   
90                         3.000   

                                                       \
                                             metadata   
0   (a50628c5-4b7e-341e-9091-7b0177228c9e, 9317815...   
1   (a50628c5-4b7e-341e-9091-7b0177228c9e, 9317815...   
2   (a50628c5-4b7e-341e-9091-7b0177228c9e, 9317815...   
3   (a50628c5-4b7e-341e-9091-7b0177228c9e, 9317815...   
4   (a50628c5-4b7e-341e-9091-7b0177228c9e, 9317815...   
5   (a50628c5-4b7e-341e-9091-7b0177228c9e, df95efe...   
6   (a50628c5-4b7e-341e-9091-7b0177228c9e, df95efe...   
7   (a50628c5-4b7e-341e-9091-7b0177228c9e, df95efe...   
8   (a50628c5-4b7e-341e-9091-7b0177228c9e, df95efe...   
9   (a50628c5-4b7e-341e-9091-7b0177228c9e, df95efe...   
10  (a50628c5-4b7e-341e-9091-7b0177228c9e, df95efe...   
11  (a50628c5-4b7e-341e-9091-7b0177228c9e, df95efe...   
12  (a50628c5-4b7e-341e-9091-7b0177228c9e, df95efe...   
13  (a50628c5-4b7e-341e-9091-7b0177228c9e, 18da299...   
14  (a50628c5-4b7e-341e-9091-7b0177228c9e, 18da299...   
15  (a50628c5-4b7e-341e-9091-7b0177228c9e, 18da299...   
16  (a50628c5-4b7e-341e-9091-7b0177228c9e, 18da299...   
17  (a50628c5-4b7e-341e-9091-7b0177228c9e, 18da299...   
18  (a50628c5-4b7e-341e-9091-7b0177228c9e, 18da299...   
19  (a50628c5-4b7e-341e-9091-7b0177228c9e, 18da299...   
20  (a50628c5-4b7e-341e-9091-7b0177228c9e, 18da299...   
21  (a50628c5-4b7e-341e-9091-7b0177228c9e, 18da299...   
22  (a50628c5-4b7e-341e-9091-7b0177228c9e, 9a23a7b...   
23  (a50628c5-4b7e-341e-9091-7b0177228c9e, 9a23a7b...   
24  (a50628c5-4b7e-341e-9091-7b0177228c9e, 9a23a7b...   
25  (a50628c5-4b7e-341e-9091-7b0177228c9e, 9a23a7b...   
26  (a50628c5-4b7e-341e-9091-7b0177228c9e, 9a23a7b...   
27  (a50628c5-4b7e-341e-9091-7b0177228c9e, 9a23a7b...   
28  (a50628c5-4b7e-341e-

In [32]:
metadata.keys()

dict_keys(['a50628c5-4b7e-341e-9091-7b0177228c9e', '9317815f-acb0-36d4-9218-53449fd7802d', '0706e83b-7d74-39f2-a098-7ceac72e391e', 'df95efe4-824a-38c8-84d6-03d9656dc6a1', 'fae26437-e285-35e5-99b4-d611bb9ef5f0', '18da299d-2b54-30c6-974f-0e3572645f7a', 'da7c1015-7415-36a4-9d29-e8baace6e97c', '9a23a7bb-71da-3e58-a0a0-f65cb59ee227', '9ec4476e-61cd-3da1-80d9-247e1fe84d71', 'ed5d2058-59ae-383f-b376-c85e6d7709f0', 'e89df9d1-ff1f-35f5-b94b-5ef783c58b84', 'a14666e7-dd31-3d48-abf5-ab93c5ae5d05', 'cd674755-057d-3687-a7a8-bd2e17d673ff'])

In [22]:
data.columns.levels[0]

Index(['27 Al ppm', 'Base Concentration', 'Counter Ion',
       'Counter Ion Concentration', 'Total Aluminate Concentration'],
      dtype='object')

In [8]:
for d in data:
#     display(d)
    for r in d:
        display(r)

Total Aluminate Concentration  \
                           data   
0                         0.005   
1                         0.005   
2                         0.005   
3                         0.005   
4                         0.005   

                                                      
                                            metadata  
0  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...  
1  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...  
2  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...  
3  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...  
4  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...

Total Aluminate Concentration  \
                           data   
0                         0.005   
1                         0.005   
2                         0.005   
3                         0.005   
4                         0.005   
5                         0.005   
6                         0.005   
7                         0.005   

                                                      
                                            metadata  
0  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...  
1  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...  
2  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...  
3  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...  
4  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...  
5  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...  
6  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...  
7  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...

Total Aluminate Concentration  \
                           data   
0                         0.005   
1                         0.005   
2                         0.005   
3                         0.005   
4                         0.005   
5                         0.005   
6                         0.005   
7                         0.005   
8                         0.005   

                                                      
                                            metadata  
0  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...  
1  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...  
2  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...  
3  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...  
4  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...  
5  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...  
6  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...  
7  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...  
8  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...

Total Aluminate Concentration  \
                            data   
0                          0.050   
1                          0.050   
2                          0.050   
3                          0.050   
4                          0.050   
5                          0.050   
6                          0.050   
7                          0.050   
8                          0.050   
9                          0.050   
10                         0.050   
11                         0.100   
12                         0.100   
13                         0.100   
14                         0.100   
15                         0.100   
16                         0.100   
17                         0.100   
18                         0.100   
19                         0.100   
20                         0.100   
21                         0.100   
22                         0.396   
23                         0.396   
24                         0.396   
25                         0.396   
26                         0.396   
27                         0.396   
28                         0.396   
29                         0.396   
..                           ...   
39                         0.800   
40                         0.800   
41                         0.800   
42                         1.200   
43                         1.200   
44                         1.200   
45                         1.200   
46                         1.200   
47                         1.200   
48                         1.200   
49                         1.200   
50                         1.200   
51                         1.200   
52                         1.200   
53                         2.000   
54                         2.000   
55                         2.000   
56                         2.000   
57                         2.000   
58                         2.000   
59                         2.000   
60                         2.000   
61                         2.000   
62                         3.000   
63                         3.000   
64                         3.000   
65                         3.000   
66                         3.000   
67                         3.000   
68                         3.000   

                                                       
                                             metadata  
0   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
1   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
2   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
3   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
4   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
5   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
6   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
7   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
8   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
9   (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
10  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
11  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
12  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
13  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
14  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
15  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
16  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
17  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
18  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
19  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
20  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
21  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
22  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
23  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
24  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
25  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
26  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
27  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...  
28  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2... 

Counter Ion Concentration                                                   
                       data                                           metadata
0                      2.93  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...
1                      4.92  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...
2                      6.85  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...
3                      9.13  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...
4                     10.71  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...

...

Base Concentration                                                   
                 data                                           metadata
0               80.07  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
1               79.24  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
2               78.67  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
3               78.96  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
4               78.11  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
5               77.80  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
6               77.03  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
7               76.67  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
8               76.06  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
9               75.12  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
10              74.36  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
11              80.31  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
12              80.21  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
13              80.07  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
14              79.76  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
15              78.98  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
16              79.43  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
17              78.37  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
18              76.77  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
19              75.73  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
20              74.20  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
21              73.05  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
22              80.21  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
23              80.12  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
24              79.93  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
25              79.41  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
26              79.15  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
27              78.74  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
28              78.32  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
29              77.73  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
..                ...                                                ...
39              79.00  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
40              78.67  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
41              78.10  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
42              80.28  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
43              79.93  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
44              79.88  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
45              79.88  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
46              80.13  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
47              80.27  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
48              79.56  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
49              79.71  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
50              79.96  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
51              79.68  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
52              79.44  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
53              80.21  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
54              80.14  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
55              80.02  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
56              79.83  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
57              80.02  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
58              79.83  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
59              79.49  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
60              79.52  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e,

27 Al ppm                                                   
       data                                           metadata
0     79.90  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...
1     79.84  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...
2     79.72  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...
3     79.66  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...
4     79.66  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 2443540...

27 Al ppm                                                   
       data                                           metadata
0     79.92  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...
1     79.84  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...
2     79.66  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...
3     79.54  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...
4     79.36  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...
5     79.13  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...
6     78.83  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...
7     78.47  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, da4bf88...

27 Al ppm                                                   
       data                                           metadata
0     79.54  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...
1     79.98  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...
2     79.13  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...
3     78.45  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...
4     77.67  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...
5     76.47  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...
6     74.53  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...
7     73.14  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...
8     71.32  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, 3b53e4b...

27 Al ppm                                                   
        data                                           metadata
0       4.02  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
1       8.10  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
2       9.80  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
3       9.12  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
4      11.16  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
5      11.84  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
6      13.17  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
7      13.88  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
8      14.86  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
9      15.92  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
10     16.92  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
11      1.58  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
12      3.02  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
13      4.56  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
14      6.06  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
15      8.91  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
16      7.55  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
17     10.68  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
18     13.61  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
19     15.05  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
20     16.86  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
21     17.84  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
22      1.52  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
23      3.02  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
24      4.45  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
25      7.51  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
26      8.95  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
27     10.45  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
28     11.19  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
29     13.53  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
..       ...                                                ...
39     13.53  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
40     15.17  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
41     16.75  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
42      3.02  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
43      4.81  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
44      5.99  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
45      7.53  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
46      8.95  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
47     10.51  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
48     11.99  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
49     12.59  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
50     13.53  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
51     15.03  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
52     16.57  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
53      3.02  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
54      4.64  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
55      5.99  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
56      7.39  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
57      8.78  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
58     10.43  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
59     11.78  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
60     13.30  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
61     14.55  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
62      4.79  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
63      5.97  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
64      7.55  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
65      8.95  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
66     10.51  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
67     11.99  (9b9fac20-59f2-3bab-8dac-bb02e33ad70e, d6e32e2...
68     12.59  (9b9fac20-59f2-3bab-8

In [8]:
import bokeh as bk
import bokeh.models

In [15]:
# dir(bk.models.ColumnDataSource(data))

In [16]:
bk.models.ColumnDataSource(data).data

{'Total Aluminate Concentration_data': array([0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005]),
 'Total Aluminate Concentration_metadata': array([('98e33f3a-9b17-3618-b269-3e44d62eb7f1', '2443540d-0ae0-3239-afb8-546a68b8b01a', '7fc79927-5857-32f1-b2df-d8d63a2a2e15'),
        ('98e33f3a-9b17-3618-b269-3e44d62eb7f1', '2443540d-0ae0-3239-afb8-546a68b8b01a', '7fc79927-5857-32f1-b2df-d8d63a2a2e15'),
        ('98e33f3a-9b17-3618-b269-3e44d62eb7f1', '2443540d-0ae0-3239-afb8-546a68b8b01a', '7fc79927-5857-32f1-b2df-d8d63a2a2e15'),
        ('98e33f3a-9b17-3618-b269-3e44d62eb7f1', '2443540d-0ae0-3239-afb8-546a68b8b01a', '7fc79927-5857-32f1-b2df-d8d63a2a2e15'),
        ('98e33f3a-9b17-3618-b269-3e44d62eb7f1', '2443540d-0ae0-3239-afb8-546a68b8b01a', '7fc79927-5857-32f1-b2df-d8d63a2a2e15'),
        ('98e33f3a-9b17-3618-b269-3e44d62eb7f1', '2fe4c708-840f-3a00-a1ba-7c63385a5977', '